Question 1: Where do people stay? Which neighborhoods are the most popular ones?

Read in the necessary libraries

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# To achive an uniform appearance of all plots the 'darkgrid' template of seaborn is set as main visualization style
sns.set(style="darkgrid")

Read in the cleaned data:

In [30]:
df_raw_cleaned = pd.read_csv('airbnb_data_berlin_cleaned.csv', sep=',')

We analyze the availability of all listings in Berlin over several time frames and invert them to get the current number of days booked. We can not filter if the hosts did reserve their apartment for their own or blocked the availability for other reasons, but it should give a hint which neighborhoods are the most frequently booked ones.

Gather all data of interest out of the main dataframe of all listings, filtered for listings that are marked as 'availale' through the column 'has_availality'.

In [31]:
df_analyze_q1 = df_raw_cleaned.loc[df_raw_cleaned['has_availability'] == 't',['neighbourhood_group_cleansed', 'price', 'availability_30', 'availability_60', 'availability_90', 'availability_365']].copy()

Filter some outliers:
- zero availality over the whole year
- listings where the availabilities for all three timeframes are the same (filter hosts who only open up booking for certain periods of time)

In [32]:
# filter for not listed apartments
df_analyze_q1 = df_analyze_q1[df_analyze_q1.availability_365 != 0].copy()

In [33]:
# filter for listings bookable only for certain periods of time
print(f'Listings only bookable for certain timeframes: {df_analyze_q1.query("availability_30 == availability_60 == availability_90 == availability_365").shape[0]}')
index = df_analyze_q1.query("availability_30 == availability_60 == availability_90 == availability_365").index
df_analyze_q1 = df_analyze_q1.drop(index=index, axis=0).copy()

Listings only bookable for certain timeframes: 0


For the days a listing is already booked we need to invert the availability over the given timeframesz

In [34]:
df_analyze_q1['booked_30'] = 30 - df_analyze_q1.availability_30
df_analyze_q1['booked_60'] = 60 - df_analyze_q1.availability_60
df_analyze_q1['booked_90'] = 90 - df_analyze_q1.availability_90
df_analyze_q1['booked_365'] = 365 - df_analyze_q1.availability_365

TypeError: unsupported operand type(s) for -: 'int' and 'str'

To visualize the data we reshape the dataframe in the way that we melt all columns regarding different booking timeframes into one column. 

In [ ]:
df_plot_q1 = df_analyze_q1.melt(
    value_vars=['booked_30','booked_60','booked_90'], 
    id_vars=['neighbourhood_group_cleansed','price'],
    var_name='booking_tf',
    value_name='days_booked'
    ).copy()

And plot them as a grouped barplot:

In [ ]:
fig = sns.barplot(
    data=df_plot_q1, 
    y='neighbourhood_group_cleansed', 
    x='days_booked', 
    hue='booking_tf', 
    ci=None,
    )
fig.set_title('Average nights booked over certain timeframes in different neighbourhoods in Berlin', pad=10, fontsize=16, color='black'); 
fig.set_xlabel('Booked nights over multiple timeframes', fontsize=12, color='black', labelpad=10);
fig.set_ylabel('Neighboorhoods in Berlin', fontsize=12, color='black', labelpad=10);

fig.legend(loc=1, title='Timeframes of bookings', labels=['Bookings - 30 days','Bookings - 60 days','Bookings - 90 days'])
sns.move_legend(fig, "upper left", bbox_to_anchor=(1, 1))

The plot shows that the number of booked nights regarding different timeframes doesn't differ that much between all neighborhoods, at least regarding the 30 and 60 day timeframe. Comparing the number of booked nights over the next year though, the differences seem to be of more significance. Therefore we use this timeframe for our visualization.

In [ ]:
# order = df_analyze_q2.groupby('neighbourhood_group_cleansed')['booked_365'].mean().sort_values(ascending=False).index.to_list()

In [ ]:
df_analyze_q1_agg = df_analyze_q1.groupby('neighbourhood_group_cleansed', as_index=False).aggregate(nights_booked=('booked_365', 'mean'))

In [ ]:
fig = sns.barplot(data=df_analyze_q1_agg.sort_values(by='nights_booked', ascending=False), x='nights_booked', y='neighbourhood_group_cleansed', ci=None, color='cornflowerblue')

fig.set_title('Average nights booked over 365 days', pad=10, fontsize=16, color='black'); 
fig.set_xlabel('Booked nights over one year', fontsize=12, color='black', labelpad=10);
fig.set_ylabel('Neighborhoods in Berlin', fontsize=12, color='black', labelpad=10);